# Senior Cubers Worldwide - Weekly Competition

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/wca-ipy/

## Initialisation

Basic approach to determine the project directory

In [1]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

## Formatting Functions

Functions to convert results to and from seconds or display an age category

In [2]:
def numSeconds(value):
    '''Convert float or string to number of seconds - e.g. 1:05.31 returns 65.31'''   
    
    if isinstance(value, float):
        return round(value, 2)
    elif ':' in value:
        parts = value.split(':')
        return round(int(parts[0]) * 60 + float(parts[1]), 2)
    elif value.lower() == 'n/a':
        return -1
    else:
        return None


def formatResult(value):
    '''Convert number of seconds to displayable time - e.g. 65.31 returns 1:05.31'''
    
    if value:
        if value > 60:
            return '{:d}:{:05.2f}'.format(int(value // 60), value - int(value // 60) * 60)
        else:
            return '{:.2f}'.format(value)
    else:
        return 'DNF'

In [3]:
def formatAge(age):
    '''Format age for report'''
    
    if age < 40:
        return '<{}'.format(age + 10)
    else:
        return '{}+'.format(age)

    
def formatAgeLong(age):
    '''Format age for report'''
    
    if age < 40:
        return 'Under {}'.format(age + 10)
    else:
        return 'Over {}'.format(age)

In [4]:
def getSafeName(name):
    '''Return name which is safe for URL'''
    
    nameDecomposed = unicodedata.normalize('NFKD', name)
    return nameDecomposed.encode('ascii', 'ignore').decode('ascii').replace(' ', '_')

In [5]:
title = 'Senior Cubers Worldwide - Weekly Competition'

In [6]:
eventNames = \
[
    '3x3x3',
    '2x2x2',
    '4x4x4',
    '3BLD',
]

## Generic Class

Generic class to ensure that all custom classes are printable

In [7]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Result Class

Simple class for a competition result

In [8]:
class Result(Printable):
    def __init__(self, date, rank, name, age, result, single, average, awards, solves, link):
        self.date, self.rank = date, rank
        self.name, self.age = name, age
        self.result, self.single, self.average = result, single, average
        self.awards = awards
        self.solves, self.link = solves, link

## Event Class

Simple class to hold a list of results

In [9]:
class Event(Printable):
    def __init__(self, name):
        self.name = name
        self.results = []
        self.maxSolves = 0

    def addResult(self, result):
        self.results.append(result)

## Person Class

Results for a single person 

In [10]:
import unicodedata

class Person(Printable):
    def __init__(self, name):
        self.name = name
        self.age = 0
        self.bestSingles = {}
        self.bestAverages = {}
        self.events = {}


    def isBestSingle(self, eventName, single):
        if eventName in self.bestSingles:
            if single and self.bestSingles[eventName] >= single:
                self.bestSingles[eventName] = single
                return True
        else:
            self.bestSingles[eventName] = single

        return False
    
    
    def isBestAverage(self, eventName, average):
        if eventName in self.bestAverages:
            if average and self.bestAverages[eventName] >= average:
                self.bestAverages[eventName] = average
                return True
        else:
            self.bestAverages[eventName] = average

        return False

    
    def addResult(self, eventName, result):
        if eventName in self.events:
            event = self.events[eventName]
        else:
            event = Event(eventName)
            self.events[eventName] = event

        event.addResult(result)

        if result.age > self.age:
            self.age = result.age
            
            
    def saveEventHeader(self, f, event):
        '''Save header for table'''
        
        f.write('#### {}\n\n'.format(event.name))

        f.write('| Date | # | Age | Single | Average | Awards |')
        for i in range(event.maxSolves):
            f.write(' Solve {} |'.format(i + 1))
        f.write(' Video |\n')

        f.write('| :--: | :--: | :--: | --: | --: | :--: |')
        for i in range(event.maxSolves):
            f.write(' --: |')
        f.write(' :-- |\n')

        
    def saveEventResult(self, f, event, result):
        '''Save header for table'''
        
        link = '[{}](../{}/{}.md)'.format(result.date, event.name, result.date)
        f.write('| {} | {:d} | {} |'.format(link, result.rank, formatAge(result.age)))
        f.write(' {} |'.format(formatResult(result.single)))
        f.write(' {} |'.format(formatResult(result.average)))
        f.write(' {} |'.format(result.awards))
        for i in range(event.maxSolves):
            if len(result.solves) > i:
                if result.solves[i] and result.solves[i] < 0:
                    f.write(' - |')
                else:
                    f.write(' {} |'.format(formatResult(result.solves[i])))
            else:
                f.write(' - |')
        if 'http' in result.link:
            f.write(' [Link]({}) |'.format(result.link))
        else:
            f.write(' |')
        f.write('\n')

        
    def saveProfile(self):
        '''Save profile as markdown'''

        docsDir = os.path.join(projdir, 'docs', 'persons')
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, getSafeName(self.name) + '.md')

        with open(outFile, 'w') as f:
            
            f.write('## {}\n'.format(title))
            f.write('### {}\n\n'.format(self.name))
            
            for eventName in eventNames:
                if eventName in self.events:
                    event = self.events[eventName]
                    
                    event.maxSolves = 0
                    for result in event.results:
                        if len(result.solves) > event.maxSolves:
                            event.maxSolves = len(result.solves)

                    self.saveEventHeader(f, event)
                    for result in reversed(event.results):
                        self.saveEventResult(f, event, result)

                    f.write('\n')

## Competition Class

Results for a single competition 

In [11]:
from xlrd import open_workbook
from datetime import datetime

import Levenshtein

class Competition(Printable):
    def __init__(self, xslx):
        self.xlsx = xslx
        self.date = os.path.basename(os.path.dirname(xslx))
        self.events = {}

        
    def saveEventHeader(self, f, result):
        '''Save header for table'''
        
        f.write('| # | Name | Age | Single | Average | Awards |')
        for i in range(len(result.solves)):
            f.write(' Solve {} |'.format(i + 1))
        f.write(' Video |\n')

        f.write('| :--: | -- | :--: | --: | --: | :--: |')
        for i in result.solves:
            f.write(' --: |')
        f.write(' :-- |\n')

        
    def saveEventResult(self, f, result):
        '''Save header for table'''
        
        link = '[{}](../persons/{}.md)'.format(result.name, getSafeName(result.name))
        f.write('| {:d} | {} | {} |'.format(result.rank, link, formatAge(result.age)))
        f.write(' {} |'.format(formatResult(result.single)))
        f.write(' {} |'.format(formatResult(result.average)))
        f.write(' {} |'.format(result.awards))
        for solve in result.solves:
            if solve and solve < 0:
                f.write(' - |')
            else:
                f.write(' {} |'.format(formatResult(solve)))
        if 'http' in result.link:
            f.write(' [Link]({}) |'.format(result.link))
        else:
            f.write(' |')
        f.write('\n')

        
    def saveEvent(self, eventName):
        '''Save results as markdown'''

        event = self.events[eventName]
        results = event.results
        
        docsDir = os.path.join(projdir, 'docs', eventName)
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, self.date + '.md')

        with open(outFile, 'w') as f:
            
            f.write('## {}\n'.format(title))
            f.write('### {} Competition {}\n\n'.format(eventName, self.date))
            
            self.saveEventHeader(f, results[0])
            for result in results:
                self.saveEventResult(f, result)

        
    def saveEvents(self):
        '''Save results as markdown'''

        for eventName in self.events:
            self.saveEvent(eventName)


    def processSheets(self):
        '''Process a spreadsheet which has been downloaded from Google Sheets'''

        wb = open_workbook(self.xlsx)

        for sheet in wb.sheets():
            # Fixed columns
            indexRank = 0
            indexName = 1
            indexAge = 2
            indexResult = 3

            # Variable columns
            resultName = None
            labelAverage = None
            indexAverage = None
            indexAwards = None
            indexSolves = []
            indexLink = None
            
            # Counters, etc
            rowNo = 1
            prevResult = None
            results = []

            eventName = sheet.name
            event = Event(eventName)
            self.events[eventName] = event
            
            for row in range(sheet.nrows):
                values = []
                for col in range(sheet.ncols):
                    values.append(sheet.cell(row,col).value)

                # Process header row - must be row 3
                if rowNo == 3:
                    for i in range(len(values)):
                        if str(values[i]).startswith('Award'):
                            indexAwards = i
                        elif str(values[i]).startswith('Solve'):
                            indexSolves.append(i)
                        elif 'Mo3' in str(values[i]):
                            labelAverage = values[i]
                            indexAverage = i
                        elif 'Ao5' in str(values[i]):
                            labelAverage = values[i]
                            indexAverage = i
                        elif 'link' in str(values[i]).lower():
                            indexLink = i
                    resultName = values[indexResult]

                # Process result row - must be after header on row 3
                elif rowNo > 3:
                    # Fixed fields
                    name = values[indexName]
                    age = values[indexAge].lower()
                    if 'under' in age:
                        age = int(age.replace('under', '').replace(' ', '')) - 10
                    else:
                        age = int(age.replace('+', ''))
                    
                    # Determine result in seconds and rank
                    thisResult = numSeconds(values[indexResult])
                    if thisResult != prevResult:
                        rank = rowNo - 3

                    # Process solves - determine single, average, etc.
                    solves = []
                    single = None
                    for indexSolve in indexSolves:
                        value = numSeconds(values[indexSolve])
                        if value and value > 0:
                            if single is None or single > value:
                                single = value
                        solves.append(value)
                    average = numSeconds(values[indexAverage])
                    awards = values[indexAwards]
                    link = values[indexLink]
                    
                    # Check person
                    if name in persons:
                        person = persons[name]
                    else:
                        for person in persons:
                            if (Levenshtein.distance(person, name) < 5):
                                print("WARNING: Similar names -", person, "+", name)
                        person = Person(name)
                        persons[name] = person

                    # Check rank
                    if (thisResult and prevResult and thisResult < prevResult):
                        print('ERROR: Order is incorrect for {} in {} {}'.format(name, eventName, self.date))
                    if (values[indexRank] != rank):
                        print('ERROR: Rank is incorrect for {} in {} {}'.format(name, eventName, self.date))

                    # Check single
                    if person.isBestSingle(eventName, single):
                        if '⚡' not in awards:
                            print('WARNING: Missing ⚡ for {} in {} {}'.format(name, eventName, self.date))
                    else:
                        if '⚡' in awards:
                            print('WARNING: Spurious ⚡ for {} in {} {}'.format(name, eventName, self.date))

                    # Check average
                    if person.isBestAverage(eventName, average):
                        if '🔥' not in awards:
                            print('WARNING: Missing 🔥 for {} in {} {}'.format(name, eventName, self.date))
                    else:
                        if '🔥' in awards:
                            print('WARNING: Spurious 🔥 for {} in {} {}'.format(name, eventName, self.date))

                    # Record result
                    result = Result(self.date, rank, name, age, thisResult, single, average, awards, solves, link)
                    results.append(result)
                    person.addResult(eventName, result)
                    event.addResult(result)

                    prevResult = thisResult

                rowNo += 1

## Main Code

Process all competitions

In [12]:
import glob
import time

pc1 = time.perf_counter()

In [13]:
# Process spreadsheets

datePattern = '[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]'
xlsxPattern = '*.xlsx'

persons = {}
competitions = {}

for xlsx in glob.glob(os.path.join(projdir, 'data', datePattern, xlsxPattern)):
    competition = Competition(xlsx)
    competition.processSheets()
    competition.saveEvents()
    competitions[competition.date] = competition

In [14]:
# Save event histories
for eventName in eventNames:
    docsDir = os.path.join(projdir, 'docs', eventName)
    if not os.path.exists(docsDir):
        os.makedirs(docsDir)

    outFile = os.path.join(docsDir, 'README.md')

    with open(outFile, 'w') as f:

        f.write('## {}\n'.format(title))
        f.write('### {} Summary\n'.format(eventName))

        for competition in sorted(competitions, reverse = True):
            competition = competitions[competition]
            
            if eventName in competition.events:
                f.write('#### [{}]({}.md)\n\n'.format(competition.date, competition.date))
                results = competition.events[eventName].results
                competition.saveEventHeader(f, results[0])
                for result in results:
                    if '🥇' in result.awards or '🥈' in result.awards or '🥉' in result.awards:
                        competition.saveEventResult(f, result)
                        
            f.write('\n')

In [15]:
# Save event persons
for eventName in eventNames:
    docsDir = os.path.join(projdir, 'docs', eventName)
    if not os.path.exists(docsDir):
        os.makedirs(docsDir)

    outFile = os.path.join(docsDir, 'persons.md')

    with open(outFile, 'w') as f:

        f.write('## {}\n'.format(title))
        f.write('### {} Competitors\n\n'.format(eventName))

        for age in range(100, 0, -10):
            found = False

            for personName in sorted(persons):
                person = persons[personName]
                if person.age == age:
                    if eventName in person.bestSingles:
                        if found == False:
                            f.write('#### {}\n\n'.format(formatAgeLong(age)))
                            f.write('| Name | Single | Average |\n')
                            f.write('| -- | --: | --: |\n')
                            found = True

                        link = '[{}](../persons/{}.md)'.format(person.name, getSafeName(person.name))
                        f.write('| {} |'.format(link))
                        f.write('{} |'.format(formatResult(person.bestSingles[eventName])))
                        f.write('{} |\n'.format(formatResult(person.bestAverages[eventName])))

            if found:
                f.write('\n')

In [16]:
# Save profile pages
for person in persons:
    persons[person].saveProfile()

In [17]:
# Save index page
outFile = os.path.join(projdir, 'docs', 'README.md')
with open(outFile, 'w') as f:
    f.write('## {}\n'.format(title))
    f.write('### Weekly Competitions Archive\n')

    f.write('#### Summaries\n')

    eventList = []
    for eventName in eventNames:
        eventList.append('[{}]({}/README.md)'.format(eventName, eventName))
    f.write('Events - {}\n\n'.format(', '.join(eventList)))

    eventList = []
    for eventName in eventNames:
        eventList.append('[{}]({}/persons.md)'.format(eventName, eventName))
    f.write('Persons - {}\n\n'.format(', '.join(eventList)))
    
    f.write('#### Archives\n')

    for competition in sorted(competitions, reverse = True):
        competition = competitions[competition]
        eventList = []
        for eventName in competition.events:
            eventList.append('[{}]({}/{}.md)'.format(eventName, eventName, competition.date))
        f.write('{} - {}\n\n'.format(competition.date, ', '.join(eventList)))
    
    f.write('#### Project Code\n')

    url = 'https://github.com/Logiqx/scw-comp'
    f.write('Python code for these reports can be found at [{}]({})\n'.format(url, url))

In [18]:
pc2 = time.perf_counter()
print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Reports completed in 1.01 seconds


## All Done!